In [1]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Set-up to have matplotlib use its support for notebook inline plots
%matplotlib inline

In [3]:
btc_dfd = pd.read_csv('Binance_BTCUSDT_1h.csv')
btc_dfd

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount
0,1.669684e+12,2022-11-29 01:00:00,BTC/USDT,16178.74,16309.03,16171.17,16250.77,9098.01403,1.476206e+08,227013.0
1,1.669680e+12,2022-11-29 00:00:00,BTC/USDT,16212.18,16224.83,16100.00,16130.62,6565.59515,1.061095e+08,125147.0
2,1.669676e+12,2022-11-28 23:00:00,BTC/USDT,16220.36,16245.92,16195.63,16212.91,4925.09556,7.988443e+07,112061.0
3,1.669673e+12,2022-11-28 22:00:00,BTC/USDT,16200.56,16235.21,16195.72,16220.76,3742.40807,6.068580e+07,89180.0
4,1.669669e+12,2022-11-28 21:00:00,BTC/USDT,16256.33,16262.77,16186.00,16199.59,6780.81367,1.099914e+08,155815.0
...,...,...,...,...,...,...,...,...,...,...
46291,1.502957e+09,2017-08-17 08:00:00,BTC/USDT,4349.99,4377.85,4333.32,4360.69,0.94990,4.139700e+03,NaN
46292,1.502953e+09,2017-08-17 07:00:00,BTC/USDT,4324.35,4349.99,4287.41,4349.99,4.44000,1.924106e+04,NaN
46293,1.502950e+09,2017-08-17 06:00:00,BTC/USDT,4315.32,4345.45,4309.37,4324.35,7.23000,3.128231e+04,NaN
46294,1.502946e+09,2017-08-17 05:00:00,BTC/USDT,4308.83,4328.69,4291.37,4315.32,23.23000,1.003048e+05,NaN


In [4]:
btc_dfd['date'] = pd.to_datetime(btc_dfd['date']).dt.date
btc_clean = btc_dfd.drop(['unix', 'symbol', 'Volume BTC', 'tradecount'], axis=1)
# res_btc = btc_clean[~(btc_clean['date'] < pd.to_datetime("2018-01-01").date())]
# res_btc

In [5]:
btc = btc_clean.iloc[::-1]
btc

,date,open,high,low,close,Volume USDT
46295,2017-08-17,16199.91,16199.91,4261.32,4308.83,1.909529e+05
46294,2017-08-17,4308.83,4328.69,4291.37,4315.32,1.003048e+05
46293,2017-08-17,4315.32,4345.45,4309.37,4324.35,3.128231e+04
46292,2017-08-17,4324.35,4349.99,4287.41,4349.99,1.924106e+04
46291,2017-08-17,4349.99,4377.85,4333.32,4360.69,4.139700e+03
...,...,...,...,...,...,...
4,2022-11-28,16256.33,16262.77,16186.00,16199.59,1.099914e+08
3,2022-11-28,16200.56,16235.21,16195.72,16220.76,6.068580e+07
2,2022-11-28,16220.36,16245.92,16195.63,16212.91,7.988443e+07
1,2022-11-29,16212.18,16224.83,16100.00,16130.62,1.061095e+08


In [6]:
btc = btc[['open', 'high', 'low', 'Volume USDT', 'close']]

In [7]:
model_data = btc.reset_index(drop=True)
model_data

,open,high,low,Volume USDT,close
0,16199.91,16199.91,4261.32,1.909529e+05,4308.83
1,4308.83,4328.69,4291.37,1.003048e+05,4315.32
2,4315.32,4345.45,4309.37,3.128231e+04,4324.35
3,4324.35,4349.99,4287.41,1.924106e+04,4349.99
4,4349.99,4377.85,4333.32,4.139700e+03,4360.69
...,...,...,...,...,...
46291,16256.33,16262.77,16186.00,1.099914e+08,16199.59
46292,16200.56,16235.21,16195.72,6.068580e+07,16220.76
46293,16220.36,16245.92,16195.63,7.988443e+07,16212.91
46294,16212.18,16224.83,16100.00,1.061095e+08,16130.62


In [8]:
btc.duplicated().sum()

94

In [9]:
duplicates = btc[btc.duplicated()]
print(duplicates)

           open      high       low  Volume USDT     close
45802   4619.43   4619.43   4619.43          0.0   4619.43
45801   4619.43   4619.43   4619.43          0.0   4619.43
45800   4619.43   4619.43   4619.43          0.0   4619.43
45799   4619.43   4619.43   4619.43          0.0   4619.43
45798   4619.43   4619.43   4619.43          0.0   4619.43
...         ...       ...       ...          ...       ...
24304  10148.93  10148.93  10148.93          0.0  10148.93
24303  10148.93  10148.93  10148.93          0.0  10148.93
22732   7480.78   7480.78   7480.78          0.0   7480.78
21196   8971.80   8971.80   8971.80          0.0   8971.80
21195   8971.80   8971.80   8971.80          0.0   8971.80

[94 rows x 5 columns]


In [10]:
# No duplicated data

In [11]:
btc.isnull().sum()

open           0
high           0
low            0
Volume USDT    0
close          0
dtype: int64

In [12]:
# No missing value

In [13]:
scaler = MinMaxScaler()
close_df = scaler.fit_transform(np.array(model_data))

In [14]:
close_df

array([[2.02095163e-01, 2.00604239e-01, 2.20056041e-02, 6.35316446e-05,
        2.11494568e-02],
       [2.11489966e-02, 2.08734292e-02, 2.24634447e-02, 3.33722706e-05,
        2.12482171e-02],
       [2.12477547e-02, 2.11271764e-02, 2.27376920e-02, 1.04078918e-05,
        2.13856293e-02],
       ...,
       [2.02406350e-01, 2.01300833e-01, 2.03836293e-01, 2.65782331e-02,
        2.02297386e-01],
       [2.02281875e-01, 2.00981529e-01, 2.02379278e-01, 3.53035421e-02,
        2.01045154e-01],
       [2.01773020e-01, 2.02256321e-01, 2.03463622e-01, 4.91146513e-02,
        2.02873513e-01]])

In [15]:
training_size = int(len(close_df)*0.8)
test_size = len(close_df)-training_size
train_data,test_data = close_df[0:training_size,:],close_df[training_size:len(close_df),:]
print('train_data: ', train_data.shape)
print('test_data: ', test_data.shape)

train_data:  (37036, 5)
test_data:  (9260, 5)


In [16]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX = []
    dataY = []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step)]  
        b = dataset[i+time_step][4]
        # print(a)
        # print(type(b))
        dataX.append(a)
        dataY.append(float(b))
    
    return np.array(dataX), np.array(dataY)

In [17]:
time_step = 10
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)
# reshape input to be [samples, time steps, features] which is required for LSTM
# X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
# X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (37025, 10, 5)
X_test:  (9249, 10, 5)


In [30]:
X_train

array([[[2.02095163e-01, 2.00604239e-01, 2.20056041e-02, 6.35316446e-05,
         2.11494568e-02],
        [2.11489966e-02, 2.08734292e-02, 2.24634447e-02, 3.33722706e-05,
         2.12482171e-02],
        [2.12477547e-02, 2.11271764e-02, 2.27376920e-02, 1.04078918e-05,
         2.13856293e-02],
        ...,
        [2.34493455e-02, 2.32458743e-02, 2.45344690e-02, 4.01750790e-05,
         2.29522501e-02],
        [2.29517507e-02, 2.27034065e-02, 2.42861228e-02, 3.39783657e-05,
         2.27042081e-02],
        [2.27037141e-02, 2.28463285e-02, 2.42861228e-02, 4.54198898e-05,
         2.34346384e-02]],

       [[2.11489966e-02, 2.08734292e-02, 2.24634447e-02, 3.33722706e-05,
         2.12482171e-02],
        [2.12477547e-02, 2.11271764e-02, 2.27376920e-02, 1.04078918e-05,
         2.13856293e-02],
        [2.13851639e-02, 2.11959122e-02, 2.24031103e-02, 6.40166505e-06,
         2.17758008e-02],
        ...,
        [2.29517507e-02, 2.27034065e-02, 2.42861228e-02, 3.39783657e-05,
        

In [19]:
y_train

array([0.02361451, 0.02181156, 0.02183956, ..., 0.94589383, 0.94774836,
       0.94726461])

In [20]:
train_X = X_train
train_y = y_train
test_X = X_test
test_y = y_test

In [21]:
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(37025, 10, 5) (37025,) (9249, 10, 5) (9249,)


## Model Training

In [22]:
hyperparams = []
batch = [16, 32, 64]
epoch = [25, 50, 100]
neuron = [50, 60, 100]
for j in batch:
    for k in epoch:
        for l in neuron:
            hyperparams.append((j,k,l))
hyperparams

[(16, 25, 50),
 (16, 25, 60),
 (16, 25, 100),
 (16, 50, 50),
 (16, 50, 60),
 (16, 50, 100),
 (16, 100, 50),
 (16, 100, 60),
 (16, 100, 100),
 (32, 25, 50),
 (32, 25, 60),
 (32, 25, 100),
 (32, 50, 50),
 (32, 50, 60),
 (32, 50, 100),
 (32, 100, 50),
 (32, 100, 60),
 (32, 100, 100),
 (64, 25, 50),
 (64, 25, 60),
 (64, 25, 100),
 (64, 50, 50),
 (64, 50, 60),
 (64, 50, 100),
 (64, 100, 50),
 (64, 100, 60),
 (64, 100, 100)]

In [27]:
hyperparam1 = hyperparams[:9]
hyperparam2 = hyperparams[9:18]
hyperparam3 = hyperparams[18:27]
hyperparam1

[(16, 25, 50),
 (16, 25, 60),
 (16, 25, 100),
 (16, 50, 50),
 (16, 50, 60),
 (16, 50, 100),
 (16, 100, 50),
 (16, 100, 60),
 (16, 100, 100)]

In [24]:
hyperparam2

[(32, 25, 50),
 (32, 25, 60),
 (32, 25, 100),
 (32, 50, 50),
 (32, 50, 60),
 (32, 50, 100),
 (32, 100, 50),
 (32, 100, 60),
 (32, 100, 100)]

In [25]:
hyperparam3

[(64, 25, 50),
 (64, 25, 60),
 (64, 25, 100),
 (64, 50, 50),
 (64, 50, 60),
 (64, 50, 100),
 (64, 100, 50),
 (64, 100, 60),
 (64, 100, 100)]

In [ ]:
%run Evaluation.ipynb

In [26]:
lstms1 = []
models1 = []
for batch, epoch, neuron in hyperparam1:
    model = Sequential()
    model.add(LSTM(neuron, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(units=1))
    model.compile(loss='mse',optimizer='adam')
    lstm = model.fit(train_X, train_y, epochs=epoch, batch_size=batch, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    lstms1.append(lstm)
    models1.append(model)

Metal device set to: Apple M1


2023-02-16 13:08:01.889842: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-16 13:08:01.890429: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/25


2023-02-16 13:08:02.175188: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-16 13:08:02.647370: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-16 13:08:02.737415: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-16 13:08:02.877499: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-16 13:08:21.906680: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-16 13:08:21.934885: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2315/2315 - 22s - loss: 8.7647e-05 - val_loss: 0.0251 - 22s/epoch - 10ms/step
Epoch 2/25
2315/2315 - 19s - loss: 9.9107e-05 - val_loss: 0.0086 - 19s/epoch - 8ms/step
Epoch 3/25


KeyboardInterrupt: 

In [ ]:
i=0
for x in models1:
    # make a prediction
    test_x2 = test_X
    yhat = x.predict(test_x2)
    # test_x2 = test_x2.reshape((test_x2.shape[0], test_x2.shape[2]))
    # invert scaling for forecast
    inv_yhat = np.concatenate((np.zeros((len(yhat), 4)),yhat), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    # inv_yhat = inv_yhat[:,0]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = np.concatenate((np.zeros((len(yhat), 4)), test_y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    # inv_y = inv_y[:,0]
    yhat_df = pd.DataFrame(inv_yhat, columns = ['Open','High','Low', 'Volume', 'Close'])
    y_df = pd.DataFrame(inv_y, columns = ['Open','High','Low', 'Volume', 'Close'])
    print(hyperparam1[i])
    print("Epoch: "+ str(lstms1[i].params['epochs']))
    print("Neurons: "+str(x.layers[0].units))
    # print(str(lstms[i].params))
    i = i+1
    # test_x2 = test_X
    # test_x3 = x.predict(test_x2)
    # # print(test_x3.shape)
    # # print(test_y.shape)
    # print(yhat_df)
    print('RMSE')
    print(RMSE(y_df['Close'],yhat_df['Close']))
    print('MAE')
    print(MAE(y_df['Close'],yhat_df['Close']))
    print('MAPE')
    print(MAPE(y_df['Close'],yhat_df['Close']))

In [ ]:
y_df

In [ ]:
yhat_df

In [ ]:
y_df.shape